First, we need to add the current folder to the PYTHONPATH in the code with the absolute location for the torchMPS library!

In [ ]:
import sys
sys.path.append(r"C:\Users\piche\Documents\dereck_codebase\udem_code\ift_4055\honor_project\src\TorchMPS-main")

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchmps import MPS

Now, let us first build a fully connected simple layer by layer approach to the procedure. 

We shall use the classic MNIST dataset in order to train our the FCNN, which 

In [ ]:
# Pick the hyperparameters for the FCNN
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 1
chosen_bs = 150
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9

In [ ]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True, transform = transforms.ToTensor(),  download = True )
train_iterator = torch.utils.data.DataLoader(dataset = train_set, batch_size=chosen_bs, shuffle=True)

test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)
test_iterator = torch.utils.data.DataLoader(dataset = test_set, batch_size= chosen_bs)


In [ ]:
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.lin1 = nn.Linear(784, 40)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(40, 40)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(40, 10)
    def forward(self, x):
        y = self.lin1(x)
        y = self.relu1(y)
        y = self.lin2(y)
        y = self.relu2(y)
        y = self.lin3(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.SGD(fcnn_teacher.parameters(), lr=chosen_lr, momentum=chosen_momentum)

#Training loop
for epoch in range(n_epochs_fcnn):
    for i, (x_mb, y_mb) in enumerate(train_iterator):

        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i % 1000 == 0):
            print(loss.item())


This was taken straight from the website 

In [ ]:

with torch.no_grad():
    acc_count = 0
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        y_mb_hat = fcnn_teacher(x_mb) #predict

        (wedontcare, y_mb_hat_class) = torch.max(y_mb_hat, 1)
        acc_count += (y_mb_hat_class == y_mb).sum().item()

    print("Accuracy: " + str(100* acc_count / len(test_iterator.dataset)))